In [95]:
import requests  # För att göra HTTP-förfrågningar
import pandas as pd  # För att hantera och exportera data till Excel

# Mappning av textbetyg till numeriska värden för korrekt sortering
rating_map = {
    "Overwhelmingly Positive": 5,
    "Very Positive": 4,
    "Positive": 3,
    "Mixed": 2,
    "Negative": 1,
    "Very Negative": 0,
    "Overwhelmingly Negative": -1,
    "No rating": -2  # Använd ett lågt värde för att hamna sist vid sortering
}

# Funktion för att hämta recensioner för ett spel baserat på dess appid
def get_reviews(appid):
    review_url = f"https://store.steampowered.com/appreviews/{appid}?json=1"  # API-endpoint för recensioner
    response = requests.get(review_url)  # Gör en GET-förfrågan
    if response.status_code == 200:  # Kontrollera om förfrågan lyckades
        review_data = response.json()  # Omvandla svaret till JSON-format
        if "query_summary" in review_data:  # Kontrollera om recensioner finns
            rating_text = review_data["query_summary"].get("review_score_desc", "No rating")
            rating_numeric = rating_map.get(rating_text, -2)
            return rating_text, rating_numeric
    return "No rating", -2  # Returnera standardvärden om ingen data finns

# Funktion för att hämta och bearbeta de 10 mest sålda spelen på Steam
def scrape_steam_top_games():
    url = "https://store.steampowered.com/api/storesearch/?cc=EU&l=en&sort_by=_ASC&filter=topsellers"  # API-endpoint för topplistan
    headers = {"User-Agent": "Mozilla/5.0"}  # Simulera en riktig webbläsare
    response = requests.get(url, headers=headers)  # Gör en GET-förfrågan
    
    if response.status_code != 200:  # Kontrollera om förfrågan lyckades
        print("Misslyckades med att hämta data från Steam.")
        return []
    
    data = response.json()  # Omvandla svaret till JSON-format
    if "items" not in data:  # Kontrollera om rätt data finns i svaret
        print("Ogiltig JSON-struktur.")
        return []
    
    game_list = []  # Lista för att lagra speldata
    for game in data["items"][:10]:  # Loopa igenom de 10 första spelen
        title = game.get("name", "Okänd")  # Hämta spelets namn
        price_info = game.get("price", {})  # Hämta prisinformationen
        final_price = price_info.get("final", 0)  # Hämta slutpriset
        original_price = price_info.get("initial", 0)  # Hämta ordinarie pris
        discount = (1 - (final_price / original_price)) * 100 if original_price > 0 else 0  # Beräkna rabattprocent korrekt
        discount = round(discount, 2) if discount > 0 else 0  # Runda av rabatten och säkerställ att den är korrekt
        appid = game.get("id", None)  # Hämta spelets appid
        
        # Omvandla priser från cent till EUR
        price = final_price / 100 if final_price > 0 else 0
        original_price = original_price / 100 if original_price > 0 else 0
        
        # Hämta spelets betyg från recensionerna
        rating_text, rating_numeric = get_reviews(appid) if appid else ("Ingen betyg", -2)
        
        # Lägg till spelet i listan
        game_list.append({
            "Namn": title,
            "Pris (EUR)": price,
            "Ordinarie Pris (EUR)": original_price,
            "Rabatt (%)": discount,
            "Betyg": rating_text,
            "Betyg Nummer": rating_numeric  # Återinför numeriskt betyg för sortering
        })
    
    return sorted(game_list, key=lambda x: x["Betyg Nummer"], reverse=True)  # Sortera efter numeriskt betyg

# Funktion för att spara speldata till en Excel-fil med sorteringsmöjligheter
def save_to_excel(game_list, filename="steam_topp_spel.xlsx"):
    df = pd.DataFrame(game_list)  # Skapa en Pandas DataFrame från listan
    
    # Spara flera sorterade versioner av filen
    with pd.ExcelWriter(filename) as writer:
        df.to_excel(writer, sheet_name="Original", index=False)  # Originaldata
        df.sort_values(by="Betyg Nummer", ascending=False).to_excel(writer, sheet_name="Efter Betyg", index=False)
        df.sort_values(by="Rabatt (%)", ascending=False).to_excel(writer, sheet_name="Efter Rabatt", index=False)
        df.sort_values(by="Pris (EUR)", ascending=True).to_excel(writer, sheet_name="Efter Pris", index=False)
        df.sort_values(by="Namn", ascending=True).to_excel(writer, sheet_name="Efter Namn", index=False)
    
    print(f"Data sparad till {filename} med sorteringsalternativ")  # Bekräftelseutskrift

# Huvudfunktion som hämtar speldata och sparar den till en Excel-fil
if __name__ == "__main__":
    games = scrape_steam_top_games()  # Hämta speldata
    save_to_excel(games)  # Spara data till Excel

Data sparad till steam_topp_spel.xlsx med sorteringsalternativ
